In [3]:
import pandas as pd
import numpy as np
from os import makedirs
from os.path import join
import requests
import shutil

In [4]:
all_cik_codes = pd.read_csv('cik_codes_with_feed_urls.csv', dtype='str')
all_cik_codes = all_cik_codes.drop('id', axis=1)
all_cik_codes.head()

,cik,companyname,entityid,primaryexchange,primarysymbol,siccode,sicdescription,feed_url,SD_url_2014,SD_url_2015,SD_url_2016,feed_retrieved_date
0,706688,AARON'S INC,2480,NYSE,AAN,7359,"Equipment Rental and Leasing, Not Elsewhere Cl...",https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/706688/...,http://www.sec.gov/Archives/edgar/data/706688/...,NaN,2016-04-30
1,824142,AAON INC,938,Nasdaq Global Market,AAON,3585,Air-Conditioning and Warm Air Heating Equipmen...,https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/824142/...,http://www.sec.gov/Archives/edgar/data/824142/...,NaN,2016-04-30
2,881890,ABAXIS INC,11929,Nasdaq Global Market,ABAX,3829,"Measuring and Controlling Devices, Not Elsewhe...",https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/881890/...,http://www.sec.gov/Archives/edgar/data/881890/...,NaN,2016-04-30
3,1551152,ABBVIE INC.,883230,NYSE,ABBV,2834,Pharmaceutical Preparations,https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/1551152...,http://www.sec.gov/Archives/edgar/data/1551152...,NaN,2016-04-30
4,1800,ABBOTT LABORATORIES,5054,NYSE,ABT,2834,Pharmaceutical Preparations,https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/1800/00...,http://www.sec.gov/Archives/edgar/data/1800/00...,NaN,2016-04-30


In [5]:
import string
exclude = set(string.punctuation)
def clean_names(sub):
    sub['name_upper'] = sub['CompanyName'].str.upper()
    sub['name_clean'] = sub.name_upper.apply(lambda s: ''.join(ch for ch in s if ch not in exclude))
    return sub

def extract_cik_from_filing_url(url):
    try:
        return url.split('www.sec.gov/Archives/edgar/data/')[1].split('/')[0]
    except:
        print(url)


rsn_2015_scores = pd.read_csv('/Users/caged/Dropbox/RSN Mining/RSN 2015 Scores - MachineReadable.csv', dtype='str')
rsn_2015_scores = rsn_2015_scores.sort_values(by='CompanyName')
rsn_2015_scores = clean_names(rsn_2015_scores)
# I did some checking to make sure this was reasonable
rsn_2015_scores['extracted_cik_2015'] = rsn_2015_scores['2015_filing'].apply(extract_cik_from_filing_url)
print(len(rsn_2015_scores))
rsn_2015_scores.head()

155


,UID,CompanyName,MTD14_Pilot,Industry,Sector,Peer Group,2015_filing,2014_filing,MA1_1A_1,MA1_1A_2,...,MA5_19_3,MA5_19_4,MA5_19_5,MA5_19_2,MA5_19_1,MA5_20_1,MA5_20_2,name_upper,name_clean,extracted_cik_2015
95,116,3M Co.,1,Industrial Conglomerates,Industrials,Industrial Conglomerates,http://www.sec.gov/Archives/edgar/data/66740/0...,http://www.sec.gov/Archives/edgar/data/66740/0...,1,1,...,NaN,NaN,NaN,1,NaN,0,NaN,3M CO.,3M CO,66740
86,105,ABB LTD,1,Electrical Equipment,Industrials,Electrical Equipment,http://www.sec.gov/Archives/edgar/data/1091587...,http://www.sec.gov/Archives/edgar/data/1091587...,1,1,...,NaN,NaN,NaN,1,NaN,0,0,ABB LTD,ABB LTD,1091587
130,157,ASML Holding NV,NaN,Semiconductors and Semiconductor Equipment,Information Technology,Semiconductors and Semiconductor Equipment,http://www.sec.gov/Archives/edgar/data/937966/...,http://www.sec.gov/Archives/edgar/data/937966/...,1,1,...,NaN,NaN,NaN,1,1,0,NaN,ASML HOLDING NV,ASML HOLDING NV,937966
67,82,Abbott Laboratories,1,Healthcare Equipment and Supplies,Healthcare,Healthcare Equipment and Supplies,http://www.sec.gov/Archives/edgar/data/1800/00...,http://www.sec.gov/Archives/edgar/data/1800/00...,0,0,...,NaN,NaN,NaN,1,NaN,0,NaN,ABBOTT LABORATORIES,ABBOTT LABORATORIES,1800
93,112,"Acuity Brands, Inc.",NaN,Electrical Equipment,Industrials,Electrical Equipment,http://www.sec.gov/Archives/edgar/data/1144215...,http://www.sec.gov/Archives/edgar/data/1144215...,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACUITY BRANDS, INC.",ACUITY BRANDS INC,1144215


In [6]:
rsn_2015_scores['2015_filing'][0]

'http://www.sec.gov/Archives/edgar/data/53669/000005366915000022/0000053669-15-000022-index.htm'

In [8]:
all_cik_codes.SD_url_2014[0]

'http://www.sec.gov/Archives/edgar/data/706688/000070668814000029/0000706688-14-000029-index.htm'

In [ ]:
names_match = rsn_2015_scores.merge(all_cik_codes, left_on='extracted_cik_2015', right_on='cik', how='left')
print(len(names_match))
names_match.head()

In [ ]:
names

In [ ]:
print('No simple match: ', sum(pd.isnull(names_match.siccode)))
print('Simple match: ', sum(~pd.isnull(names_match.siccode)))

### There are 29 companies that don't match


In [ ]:
match = names_match[~pd.isnull(names_match.feed_url)]
no_match = names_match[pd.isnull(names_match.feed_url)]
no_match.head()

## Now lets get the actual docs

In [ ]:
disclosure_docs = pd.read_csv('disclosure_docs_list.csv', dtype='str')
disclosure_docs = disclosure_docs.drop('Unnamed: 0', axis=1)
disclosure_docs = disclosure_docs.rename(columns={'req_cik': 'req_company_name', 'req_company_name': 'req_cik'})
disclosure_docs.ret_filing_date = pd.to_datetime(disclosure_docs.ret_filing_date)
disclosure_docs.head()

In [ ]:
disclosure_docs_2015 = disclosure_docs[disclosure_docs.ret_filing_date.dt.year == 2015]
disclosure_docs_2015.head()

In [ ]:
match.head()

In [ ]:
match.to_csv('/Users/caged/Dropbox/RSN Mining/indicator_data_matched_with_cik_data.csv')

In [ ]:
## EXPENSIVE

# Save all the data for the matched rows
for i, company in match.iterrows():
    print(i)
    cik = company.cik
    dir_name = ' '.join([company.primarysymbol, '--', cik])
    dir_path = join('supporting_docs_2015', dir_name)
    makedirs(dir_path)
    relevant_docs = disclosure_docs_2015[disclosure_docs_2015.req_cik == cik]
    for j, doc in relevant_docs.iterrows():
        file_path = join(dir_path, doc.ret_description)
        r = requests.get(doc.ret_url, stream=True)
        if r.status_code == 200:
            with open(file_path, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f) 

## We have now downloaded all the SEC disclosures for the companies in `indicator_data_matched_with_cik_data.csv`

```
    cik = company.cik
    dir_name = ' '.join([company.primarysymbol, '--', cik])
    dir_path = join('supporting_docs_2015', dir_name)
    relevant_docs = disclosure_docs_2015[disclosure_docs_2015.req_cik == cik]
    for j, doc in relevant_docs.iterrows():
        file_path = join(dir_path, doc.ret_description)
```